In [1]:
ShmekelPath = r'C:\Users\edenp\PycharmProjects\Shmekel'
template_path = r'configs/eden/classification_refinfoce_loss_grid_search/fine_tune_0/template.json'

import sys
sys.path.insert(0, ShmekelPath)
sys.executable

import os
os.chdir(ShmekelPath)

import pandas
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
from pandas.api.types import is_string_dtype, is_numeric_dtype
import pandas as pd
import numpy as np
from api.core.grid_search import GridSearch


from Utils.logger import logger
logger.set_stream_level('warning')



Using TensorFlow backend.


In [9]:
THE_METRIC = 'best epoch values by val_acc'


def _generate_plotly_dim_dict(df: pandas.DataFrame, field):
    dim_dict = {}
    dim_dict["label"] = field
    column = df[field]
    if is_numeric_dtype(column):
        dim_dict["values"] = column
    elif is_string_dtype(column):
        texts = column.unique()
        dim_dict["values"] = [
            np.argwhere(texts == x).flatten()[0] for x in column
        ]
        dim_dict["tickvals"] = list(range(len(texts)))
        dim_dict["ticktext"] = texts
    else:
        raise Exception("Unidentifiable Type")

    return dim_dict


def data_frame_to_parallel_plot_data(df: pandas.DataFrame):
    line = dict(color=-df[THE_METRIC], colorscale='Jet')
    dimensions = [_generate_plotly_dim_dict(df, field) for field in df]

    return [go.Parcoords(line=line, dimensions=dimensions)]


def grid_search_to_data_frame(gs: GridSearch, the_metric=THE_METRIC, metrics=('val_certainty_sharpness', 'val_uncertain_fraction')):
    converter = gs.get_exp_name_2_combs_converter(minimal=True)

    columns = None
    for comb in converter.values():
        columns = [field for field in comb.keys()]
        break

    columns = ('__Name__',) + tuple([c for c in columns if c not in ['__Name__']]) + tuple(metrics) + (the_metric,)

    df = {field: [] for field in columns}

    for exp in gs:
        line = converter[exp.name]
        
        epoch = exp.results.get_best_epoch(the_metric)
        epoch = exp.results.epoch_list[3]
        print(exp.name, epoch['val_uncertain_fraction'])
        for m in metrics + (the_metric,):
            line[m] = epoch[m]

        for field, val in line.items():
            if isinstance(val, list):
                val = str(val)
            
            df[field].append(val)
    
#     _ = df.pop('__InputShape__')
    
    return pandas.DataFrame.from_dict(df)


# gs = GridSearch(template_path=template_path)
ShmekelPath = r'C:\Users\edenp\PycharmProjects'
results_path = os.path.join(ShmekelPath, 'Shmekel_Results', 'version-0.0.1', 'Results', 'grid_results')
print(results_path)
df = pd.read_csv(results_path)
df_fields = []
for col in df.columns: 
    df_fields.append(col)

df_plot = df.loc[:,['num_of_layers', 'num_of_rnn_layers', 'best epoch number by val_acc', 'best epoch values by val_acc']]
fig = data_frame_to_parallel_plot_data(df_plot)
plotly.offline.iplot(fig)
# dim_dict = _generate_plotly_dim_dict(df, field=df_fields[0])
# print(dim_dict)



C:\Users\edenp\PycharmProjects\Shmekel_Results\version-0.0.1\Results\grid_results


In [ ]:
 %debug

In [ ]:
plotly.offline.iplot(data_frame_to_parallel_plot_data(grid_search_to_data_frame(gs)))

In [ ]:
list(gs.iter_exps())[0].results.history.keys()